In [10]:
from pathlib import Path
import IPython
import libsbgnpy.libsbgn as libsbgn
from IPython.core.display import HTML
from IPython.display import Image
from libsbgnpy.libsbgnTypes import ArcClass, GlyphClass, Language, Orientation
from pygments import highlight
from pygments.formatters import HtmlFormatter
from pygments.lexers import PythonLexer
from combine_notebooks import RESULTS_DIR

In [25]:
def pprint_xml(xml_str: str) -> None:
    """Create highlighted xml."""
    IPython.display.display(
        HTML(
            '<style type="text/css">{}</style>{}'.format(
                HtmlFormatter().get_style_defs(".highlight"),
                highlight(xml_str, PythonLexer(), HtmlFormatter()),
            )
        )
    )

In [40]:
# create empty sbgn
sbgn: libsbgn.sbgn = libsbgn.sbgn()
#sbgn_path = RESULTS_DIR / "hello_world.sbgn"
sbgn_path = "hello_world.sbgn"

# create map, set language and set in sbgn
map = libsbgn.map()
map.set_language(Language.PD)
sbgn.set_map(map)

# create some glyphs
# glyphs with labels
g = libsbgn.glyph(class_=GlyphClass.MACROMOLECULE, id="glyph4")
g.set_label(libsbgn.label(text="Hexokinase"))
g.set_bbox(libsbgn.bbox(x=110.0, y=10.0, w=90.0, h=40.0))
map.add_glyph(g)

g = libsbgn.glyph(class_=GlyphClass.SIMPLE_CHEMICAL, id="glyph3")
g.set_label(libsbgn.label(text="Glucose-6P"))
g.set_bbox(libsbgn.bbox(x=210.0, y=95.0, w=90.0, h=40.0))
map.add_glyph(g)

g = libsbgn.glyph(class_=GlyphClass.SIMPLE_CHEMICAL, id="glyph1")
g.set_label(libsbgn.label(text="ADP"))
g.set_bbox(libsbgn.bbox(x=190.0, y=150.0, w=60.0, h=60.0))
map.add_glyph(g)

g = libsbgn.glyph(class_=GlyphClass.SIMPLE_CHEMICAL, id="glyph0")
g.set_label(libsbgn.label(text="Glucose"))
g.set_bbox(libsbgn.bbox(x=10.0, y=95.0, w=90.0, h=40.0))
map.add_glyph(g)

g = libsbgn.glyph(class_=GlyphClass.SIMPLE_CHEMICAL, id="glyph2")
g.set_label(libsbgn.label(text="ATP"))
g.set_bbox(libsbgn.bbox(x=60.0, y=150.0, w=60.0, h=60.0))
map.add_glyph(g)

g = libsbgn.glyph(class_=GlyphClass.PROCESS, id="glyph5")
g.set_bbox(libsbgn.bbox(x=143.0, y=103.0, w=24.0, h=24.0))
g.add_port(libsbgn.port(y="115.0", x="179.0", id="glyph5.2"))
g.add_port(libsbgn.port(y="115.0", x="131.0", id="glyph5.1"))
map.add_glyph(g)

# Add arcs linking the glyphs together
a = libsbgn.arc(
    class_=ArcClass.CATALYSIS,
    target="glyph5",
    source="glyph4",
    id="arc0",
)
a.set_start(libsbgn.startType(y="50.0", x="155.0"))
a.set_end(libsbgn.endType(y="103.0", x="155.0"))
map.add_arc(a)

a = libsbgn.arc(
    class_=ArcClass.PRODUCTION,
    target="glyph3",
    source="glyph5.2",
    id="arc3",
)
a.set_start(libsbgn.startType(y="115.0", x="179.0"))
a.set_end(libsbgn.endType(y="115.0", x="210.0"))
map.add_arc(a)

a = libsbgn.arc(
    class_=ArcClass.PRODUCTION,
    target="glyph1",
    source="glyph5.2",
    id="arc4",
)
a.set_start(libsbgn.startType(y="115.0", x="179.0"))
a.set_end(libsbgn.endType(y="158.78679", x="198.78679"))
map.add_arc(a)

a = libsbgn.arc(
    class_=ArcClass.CONSUMPTION,
    target="glyph5.1",
    source="glyph0",
    id="arc1",
)
a.set_start(libsbgn.startType(y="115.0", x="100.0"))
a.set_end(libsbgn.endType(y="115.0", x="131.0"))
map.add_arc(a)

a = libsbgn.arc(
    class_=ArcClass.CONSUMPTION,
    target="glyph5.1",
    source="glyph2",
    id="arc2",
)
a.set_start(libsbgn.startType(y="154.62607", x="106.0051"))
a.set_end(libsbgn.endType(y="115.0", x="131.0"))
map.add_arc(a)


f_out = str(sbgn_path)
sbgn.write_file(f_out)

# render SBGN
from libsbgnpy import render

f_png: str = "hello_world.png"
render.render_sbgn(sbgn, image_file=f_png, file_format="png")
Image(f_png, width=300)

PermissionError: [Errno 13] Permission denied: 'C:\\Users\\HARVEY~1\\AppData\\Local\\Temp\\tmpp0w76g0f.sbgn'

'c:\\users\\harvey mannering\\documents\\compbiolibs\\original_repo\\combine-notebooks\\results'